<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#图像旋转指定角度" data-toc-modified-id="图像旋转指定角度-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>图像旋转指定角度</a></span></li><li><span><a href="#检查图像是否竖屏" data-toc-modified-id="检查图像是否竖屏-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>检查图像是否竖屏</a></span></li><li><span><a href="#文本倾斜校正" data-toc-modified-id="文本倾斜校正-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>文本倾斜校正</a></span></li><li><span><a href="#KNN-进行数据分类" data-toc-modified-id="KNN-进行数据分类-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>KNN 进行数据分类</a></span></li><li><span><a href="#实战-(使用KNN进行手写数字识别)" data-toc-modified-id="实战-(使用KNN进行手写数字识别)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>实战 (使用KNN进行手写数字识别)</a></span></li><li><span><a href="#K值聚类" data-toc-modified-id="K值聚类-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>K值聚类</a></span><ul class="toc-item"><li><span><a href="#仅有一个特征的数据" data-toc-modified-id="仅有一个特征的数据-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>仅有一个特征的数据</a></span></li><li><span><a href="#含有多个特征的数据" data-toc-modified-id="含有多个特征的数据-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>含有多个特征的数据</a></span></li><li><span><a href="#实战-(使用K-Means进行颜色量化)" data-toc-modified-id="实战-(使用K-Means进行颜色量化)-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>实战 (使用K-Means进行颜色量化)</a></span></li></ul></li></ul></div>

## 图像旋转指定角度
> 通过`仿射变换`进行图像旋转

In [ ]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)

    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # 计算输出图像的大小, 格式为图像的(宽, 高)
    # 宽对应图像的列数; 高对应图像的行数
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

## 检查图像是否竖屏
> 使用霍夫变换检测直线, 通过比较水平直线和垂直直线的数量来推断横竖屏

In [ ]:
def check_vertical_screen(img):

    """
    :param img:  灰度图片对象
    :return:  True(竖屏), False(横屏)
    """

    drawing = np.zeros(img.shape[:], dtype=np.uint8)

    # 使用 Laplacian算子 计算图像梯度
    img = cv2.Laplacian(img, -1, ksize=7)

    # 使用霍夫变换检测直线 (用霍夫线变换之前, 首先要对图像进行边缘检测的处理，也即霍夫线变换的直接输入只能是边缘二值图像)
    lines = cv2.HoughLinesP(img, 0.8, np.pi/180, 90,  minLineLength=300, maxLineGap=10)

    rows = 0
    cols = 0

    for line in lines:
        x1, y1, x2, y2 = line[0]

        # 水平直线
        if y1 == y2:
            cv2.line(drawing, (x1, y1), (x2, y2), (0, 255, 0), 1, lineType=cv2.LINE_AA)
            rows += 1

        # 垂直直线
        if x1 == x2:
            cv2.line(drawing, (x1, y1), (x2, y2), (0, 0, 255), 1, lineType=cv2.LINE_AA)
            cols += 1

    # 水平直线数量大于垂直直线数量, 则推断手机处于竖屏状态
    if rows > cols:
        # 竖屏
        return True
    else:
        # 横屏
        return False

## 文本倾斜校正
> 通过仿射变换对倾斜角度进行校正

In [ ]:
#-*- coding:utf-8 -*-

import cv2
import numpy as np

'''
https://mp.weixin.qq.com/s/3ERz-AS08ybfTZ46RJXxrw
'''

image = cv2.imread("imageTextR.png")
img = image.copy()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 进行图像反转(即将文字变成浅色的而背景变成深色的)
# 在计算机上执行图像操作的时候，一般前景色设为浅色，而背景(图像中我们不关心的部分)设为黑色
gray = cv2.bitwise_not(gray)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# `thresh > 0` 提取二值化图像上像素值大于0(也即255)的点
# 目的: 找出二值图像中包含前景色部分的(x, y)坐标
coords = np.column_stack(np.where(thresh > 0))

# 通过计算最小外接矩阵获取文本旋转角度 (返回一个[-90, 0)区间内的角度)
angle = cv2.minAreaRect(coords)[-1]


if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle

# 确定图像的中心坐标
(h, w) = image.shape[:2]
center = (w // 2, h // 2)

# 通过中心坐标和旋转角度计算旋转矩阵M
M = cv2.getRotationMatrix2D(center, angle, 1.0)

# 通过仿射变换对倾斜角度进行校正
rotated = cv2.warpAffine(image, M, (w, h),
    flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle),
    (10, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

cv2.imshow('image1', image)
cv2.imshow('image2', rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

## KNN 进行数据分类
> k-Nearest Neighbour 是监督学习分类器, 可以进行多类别分类

In [ ]:
#-*- coding:utf-8 -*-

import cv2
import numpy as np
import matplotlib.pyplot as plt


# 准备训练数据 (30, 2) => 30组训练数据, 每组数据有两个元素
trainData = np.random.randint(0, 100, (30, 2)).astype(np.float32)
# 准备标签[训练数据分为`3`类] (30, 1) => 30组标签数据, 每组标签数据有一个元素
responses = np.random.randint(0, 3, (30, 1)).astype(np.float32)

# 展示训练数据
red = trainData[responses.ravel() == 0]
plt.scatter(red[:, 0], red[:, 1], 80, 'r', '^')

green = trainData[responses.ravel() == 1]
plt.scatter(green[:, 0], green[:, 1], 80, 'g', 'd')

blue = trainData[responses.ravel() == 2]
plt.scatter(blue[:, 0], blue[:, 1], 80, 'b', 's')


# 准备测试数据 (5, 2) => 5组测试数据, 每组数据有两个元素
newcomer = np.random.randint(0, 100, (5, 2)).astype(np.float32)
plt.scatter(newcomer[:, 0], newcomer[:, 1], 80, 'y', 'o')

# 进行 KNN 分类
knn = cv2.ml.KNearest_create()
knn.train(trainData, cv2.ml.ROW_SAMPLE, responses)
ret, results, neighbours, dist = knn.findNearest(newcomer, k=5)

print 'result: {}\n'.format(results)
print 'neighbours: {}\n'.format(neighbours)
print 'distance: {}\n'.format(dist)

plt.show()

## 实战 (使用KNN进行手写数字识别)

In [ ]:
#-*- coding:utf-8 -*-

import cv2
import numpy as np
import matplotlib.pyplot as plt


# 图片中有 50000 个手写数字 (每个数字重复 500 遍)
# 每个数字是一个 20x20 的小图
img = cv2.imread("../imgs/digits.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# print gray.shape        # (1000, 2000)

# np.vsplit(gray, 50) ==> 50 = 1000(图片高度) / 20(数字高度)
# np.hsplit(row, 100) ==> 100 = 2000(图片宽度) / 20(数字宽度)
cells = [np.hsplit(row, 100) for row in np.vsplit(gray, 50)]
x = np.array(cells)
# print x.shape             # (50, 100, 20, 20)

# 使用每个数字的前 250 个样本做训练数据
train = x[:, :50].reshape(-1, 400).astype(np.float32)       # Size = (2500, 400)
# 使用每个数字的后 250 个样本做测试数据
test = x[:, 50:100].reshape(-1, 400).astype(np.float32)     # Size = (2500, 400)

# 准备训练标签
num = np.arange(10)
train_labels = np.repeat(num, 250)[:, np.newaxis]   # Size = (2500, 1)
test_labels = train_labels.copy()                   # Size = (2500, 1)

# KNN分类
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbours, dist = knn.findNearest(test, k=5)

# 分析结果
matches = result == test_labels
correct = np.count_nonzero(matches)
accuracy = correct * 100.0 / result.size
print 'accuracy={}'.format(accuracy)


# 保存测试数据
np.savez("knn_ocr_data.npz", train=train, train_labels=train_labels)

# 加载测试数据
with np.load('knn_ocr_data.npz') as data:
    print data.files        # ['train_labels', 'train']
    print data['train']
    print data['train_labels']

## K值聚类 

### 仅有一个特征的数据

In [ ]:
#-*- coding:utf-8 -*-

import cv2
import numpy as np
import matplotlib.pyplot as plt


x = np.random.randint(25, 100, 25)
y = np.random.randint(175, 255, 25)
z = np.hstack((x, y))
z = z.reshape((50, 1))
z = np.float32(z)

# criteria = (type, max_iter=10, epsilon=1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

# kmeans(data, K, bestLabels, criteria, attempts, flags, centers=None)
## data: 应该是 np.float32 类型的数据，每个特征应该放在一列
## K： 聚类的最终数目
## criteria: 止迭代的条件。当条件满足时，算法的迭代终止。是一个三元组，(type, max_iter，epsilon）
## attempts: 使用不同的起始标记来执行算法的次数。算法会返回紧密度最好的标记
## flags: 用来设置如何选择起始重心
compactness, labels, centers = cv2.kmeans(z, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
# compactness：紧密度，返回每个点到相应重心的距离的平方和
# labels： 标志数组，每个成员被标记为 0， 1等
# centers：由聚类的中心组成的数组
print centers   # [[59.199997], [216.56]]


A = z[labels == 0]
B = z[labels == 1]
plt.hist(A, 256, [0, 256], color='r')
plt.hist(B, 256, [0, 256], color='b')
plt.hist(centers, 32, [0, 256], color='y')
plt.show()

### 含有多个特征的数据

In [ ]:
#-*- coding:utf-8 -*-

import cv2
import numpy as np
import matplotlib.pyplot as plt

X = np.random.randint(25, 50, (25, 2))
Y = np.random.randint(60, 85, (25, 2))
Z = np.vstack((X, Y))
Z = np.float32(Z)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
compactness, labels, centers = cv2.kmeans(Z, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

A = Z[labels.ravel() == 0]
B = Z[labels.ravel() == 1]

plt.scatter(A[:,0],A[:,1])
plt.scatter(B[:,0],B[:,1],c = 'r')
plt.scatter(centers[:,0],centers[:,1],s = 80,c = 'y', marker = 's')
plt.xlabel('Height'),plt.ylabel('Weight')
plt.show()

### 实战 (使用K-Means进行颜色量化)

In [ ]:
#-*- coding:utf-8 -*-

"""
    颜色量化: 减少图片中颜色数目 (有些设备的资源有限，只能显示很少的颜色)
    把图片数据变形成 Mx3（M是图片中像素点的数目）的向量。聚类完成后，
    用聚类中心值替换与其同组的像素值，这样结果图片就只含有指定数目的颜色了
"""

import cv2
import numpy as np

img = cv2.imread('../imgs/home.jpg')
Z = img.reshape((-1, 3))
Z = np.float32(Z)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
color_num = 4
compactness, labels, centers = cv2.kmeans(Z, color_num, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)

# labels.shape = (-1, 1), 需要调用 flatten() 进行展平
result = centers[labels.flatten()]
result = result.reshape(img.shape)

cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()